In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf

# Reddit

##### Reading the data

In [2]:
titles = pd.read_csv('Cleaned_Reddit_Titles.csv')
comments = pd.read_csv('Cleaned_Reddit_Comments.csv')
print(titles.shape)
print(comments.shape)

(14347, 4)
(356446, 4)


##### Combining the data

In [3]:
reddit_df = pd.concat([titles, comments], axis = 0)
print(reddit_df.shape)

(370793, 4)


##### Removing nulls

In [4]:
reddit_df = reddit_df[reddit_df['text'].notnull()]
reddit_df.shape

(370789, 4)

##### Removing I am a bot or deleted posts or removed posts

In [5]:
q = '''
select *
from reddit_df
where text not like '%[deleted%' and text not like '%I am a bot%' and text not like '%[remove%'
'''

reddit_df = sqldf(q)
reddit_df.shape

(351179, 4)

##### Dropping duplicates

In [6]:
reddit_df = reddit_df.drop_duplicates(subset=['id'])
reddit_df.shape

(277211, 4)

##### Cleaning dates

In [7]:
reddit_df['created_utc'] = pd.to_datetime(reddit_df['created_utc'])
reddit_df.dtypes

id                     object
created_utc    datetime64[ns]
text                   object
stock_name             object
dtype: object

# Twitter

##### Reading the data

In [8]:
twitter_df = pd.read_csv('twitter_tweets.csv')
twitter_df = twitter_df[['Tweet Id', 'Datetime','Text']]
twitter_df = twitter_df.rename({'Tweet Id':'id','Datetime':'created_utc','Text':'text'}, axis = 1)
twitter_df

,id,created_utc,text
0,1477065441117827072,2021-12-31 23:53:38+00:00,“cinnamon apple sauce is our special today and...
1,1477063759172669444,2021-12-31 23:46:57+00:00,Hey @Apple Portrait mode does not work well fo...
2,1477063349330452484,2021-12-31 23:45:19+00:00,$AAPL was the 11th most mentioned on wallstree...
3,1477058597381185537,2021-12-31 23:26:26+00:00,@MoonMark_ So if I buy 1 share of Apple stock ...
4,1477054394319335424,2021-12-31 23:09:44+00:00,"ARGH! We are not a whiskey household, so it's ..."
...,...,...,...
88187,1345311527495200768,2021-01-02 10:10:37+00:00,"Hey guys, let's do a Q&amp;A! Tweet me your st..."
88188,1345248134205640704,2021-01-02 05:58:43+00:00,@mukund @psycho_stock Here is part of the thes...
88189,1345247397350281216,2021-01-02 05:55:47+00:00,@dividendvalue @psycho_stock Good point. You m...
88190,1345074529551446018,2021-01-01 18:28:52+00:00,2021 HYPER GROWTH stock pics: $IMMR haptic tec...


##### Removing nulls

In [9]:
twitter_df = twitter_df[twitter_df['text'].notnull()]
twitter_df.shape

(88192, 3)

#### Dropping duplicates

In [10]:
twitter_df = twitter_df.drop_duplicates(subset=['id'])
twitter_df.shape

(78773, 3)

# Cleaning the dates

In [11]:
twitter_df['created_utc'] = pd.to_datetime(twitter_df['created_utc'])
twitter_df.dtypes

id                           int64
created_utc    datetime64[ns, UTC]
text                        object
dtype: object

# YouTube

In [12]:
titles = pd.read_csv('Youtube_Titles.csv')
comments = pd.read_csv('Cleaned_Youtube_Comments_new.csv')
print(titles.shape)
print(comments.shape)

(3100, 4)
(112754, 4)


In [13]:
titles = titles.rename({'Video ID':'id','Video Title':'text','Timestamp':'created_utc','query':'stock'}, axis = 1)
titles = titles[['id','created_utc','text','stock']]
comments = comments.rename({'stock_name':'stock'}, axis = 1)

##### Dropping titles duplicates

In [14]:
titles = titles.drop_duplicates(subset=['id'])

In [15]:
youtube_df = pd.concat([titles, comments], axis = 0)
youtube_df.shape

(115502, 4)

##### Cleaning the dates

In [16]:
youtube_df['created_utc'] = pd.to_datetime(youtube_df['created_utc'])

In [17]:
youtube_df.dtypes

id                          object
created_utc    datetime64[ns, UTC]
text                        object
stock                       object
dtype: object

##### Removing nulls

In [18]:
youtube_df = youtube_df[youtube_df['text'].notnull()]
youtube_df.shape

(115499, 4)

# Combining all

In [19]:
youtube_df = youtube_df[['id', 'created_utc','text']]
youtube_df['source'] = 'Youtube'
reddit_df = reddit_df[['id', 'created_utc','text']]
reddit_df['source'] = 'Reddit'
twitter_df = twitter_df[['id', 'created_utc','text']]
twitter_df['source'] = 'Twitter'

In [20]:
df = pd.concat([youtube_df,reddit_df,twitter_df], axis = 0)
df.shape

(471483, 4)

##### Removing 2022 Dates

In [21]:
q = '''
select * from df
where cast(created_utc as date) <= cast('2021-12-31' as date)
'''

df = sqldf(q)

In [22]:
print(min(df.created_utc))
print(max(df.created_utc))

2021-01-01 00:00:06.000000
2021-12-31 23:58:38.000000


In [23]:
df = df.drop(['id'], axis = 1)

##### Converting to lowercase, removing punctuations and stop words

In [24]:
#to lower case
df['text'] = df['text'].apply(lambda x: x.lower())

In [25]:
import string
#removing punctuations
df['text'] = df['text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [26]:
#removing stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gevin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
stopwordslist = stopwords.words('english')
print(stopwordslist)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [28]:
stopwordslist = stopwords.words('english')
x = 'but hello if i myself had to do it you would suffer'
def stopword_remover(x):
    new_string = ''
    x = x.split()
    for i in x:
        if i in stopwordslist:
            continue
        else:
            new_string +=i
            new_string += ' '
    return new_string

stopword_remover(x)

'hello would suffer '

In [29]:
df['text'] = df['text'].apply(lambda x: stopword_remover(x))

In [30]:
df = df.sort_values(by='created_utc').reset_index(drop=True)

In [36]:
#removing blanks
df = df[df['text'] != '']

In [37]:
df.to_csv('final_data.csv', index = False)